# Chatbot con un LLM utilizando Vector Stores - Interfaces de conversación 

En este cuaderno, vamos a adicionar funcionalidades que permitan enriquecer el contexto de los chatbot con Vector Stores

## Introducción

Una de las funcionalidades mas importantes cuando se construye un Chatbot es la capacidad de mantener el contexto de una conversación. Es importante que el LLM pueda mantener el hilo de la conversación.  

## Tokens 

Son unidades de texto o codigo fuente que los LLMs utilizan para procesar y generar respuestas. Los Token pueden ser de diferentes tipos como: 

1. Palabras
3. Caracteres
4. Otros segmentos de texto o codigo

Los token dependen entonces del metodo de tokenización utilizado. Se asigna un valor numerico o identificador y son organizados en secuencias o VECTORES que alimentan a los modelos y que de igual forma son su salida. 


## Embebidos (Embeddings) 

Los Embebidos son representaciones vectoriales de las palabras o tokens que capturan su significado semantico en un espacio dimensional ...... XD 
Mejor ...
Los embebidos son la representación o codificación de los Tokens; algunos ejemplos pueden ser sentencias, parrafos o documentos completos puestos en un espacio vectorial de muchas dimensiones donde cada una de estas dimensiones representa una caracteristica o atributo aprendido del lenguaje. 

Son entonces, la forma en la cual los LLMs comprenden el significado y las relaciones que existen en el lenguaje. 

## FAISS 

Es una libreria que nos permite realizar busquedas por similitud o agrupamiento utilizando vectores de muchas dimensiones. En otras palabras FAISS va a permitir realizar busquedas sobre los Embebidos generados para este tutorial. 

In [10]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.vectorstores import FAISS

## Dataset
En el directorio data vas a encontrar un documento llamado constitucion-politica-col.pdf, en el vamos a encontrar tan solo 2 paginas con los 10 primeros artículos. 

## Nota 
Langchain ofrece diferentes herramientas para cargar datos o documentos, puedes dar un vistazo en el siguiente enlace 
[LangChainDocumentLoaders](https://python.langchain.com/docs/modules/data_connection/document_loaders)

In [2]:
loader = PyPDFLoader("data/constitucion-politica-col.pdf")
documents_col = loader.load() #
print(f"Documentos cargados={len(documents_col)}")

Documentos cargados=2


Una vez hemos cargado los archivos debemos realizar una división del contenido para generar nuestros embeddings.
No podemos cargar un solo cuerpo de texto para generar los embeddings, debemos separar el texto en pequeños fragmentos que nuestro LLM pueda soportar 

In [28]:
docs = RecursiveCharacterTextSplitter(
    length_function = len,
    is_separator_regex = False,
    chunk_size=100, 
    chunk_overlap=20
).split_documents(documents_col)

Vemos que Langchain nos ha ayudado a dividir nuestro gran cuerpo de texto en frases mas cortas que podemos enviar a nuestro generador de Embeddings

In [29]:
print(f"Documentos generados={len(docs)}")

Documentos generados=59


In [35]:
docs[:20]

[Document(page_content='13\nDe los Principios Fundamentales  (Artículos 1-4)CONSTITUCIÓN POLÍTICA COLOMBIA\nPREÁMBULO', metadata={'source': 'data/constitucion-politica-col.pdf', 'page': 0}),
 Document(page_content='PREÁMBULO\nEL PUEBLO DE COLOMBIA,\nen ejercicio de su poder soberano, representa -', metadata={'source': 'data/constitucion-politica-col.pdf', 'page': 0}),
 Document(page_content='do por sus delegatarios a la Asamblea Nacional \nConstituyente, invocando la protección de Dios, y', metadata={'source': 'data/constitucion-politica-col.pdf', 'page': 0}),
 Document(page_content='con el fin de fortalecer la unidad de la nación y', metadata={'source': 'data/constitucion-politica-col.pdf', 'page': 0}),
 Document(page_content='asegurar a sus integrantes la vida, la convivencia,', metadata={'source': 'data/constitucion-politica-col.pdf', 'page': 0}),
 Document(page_content='el trabajo, la justicia, la igualdad, el conocimiento, la libertad y la paz, dentro de un marco', metadata={'sour

## Instrucciones 

1. Alguien debe generar nuestros embeddigs, en nuestro ejemplo vamos a utilizar AWS Bedrock para generarlos y para lograrlo debemos conectarnos con el API de AWS
2. Vamos a almacenar en memoria los datos generados

In [36]:
vectorstore_faiss_aws = None
try:
    bedrock_embeddings = BedrockEmbeddings(
        credentials_profile_name="default", region_name="us-east-1"
    )
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = bedrock_embeddings
    )
    print(f"vectorstore faiss con aws bedrock: elementos en el indice={vectorstore_faiss_aws.index.ntotal}")

except ValueError as error:
    print(f"Error conectando con el vector store: {error}")
    raise error

vectorstore faiss con aws bedrock: elementos en el indice=59


Hagamos una consulta sobre nuestro Vector Store con una busqueda por similitud, simplemente vamos a pasar un string para realizar la busqueda. 

In [37]:
query = "Colombia es un Estado social de derecho"
retrieved_docs = vectorstore_faiss_aws.similarity_search(query)
print(f" Documentos obtenidos: {len(retrieved_docs)}")
print(f" Documentos obtenidos: {retrieved_docs}")

 Documentos obtenidos: 4
 Documentos obtenidos: [Document(page_content='Artículo 1 °. Colombia es un Estado social \nde derecho, organizado en forma de República', metadata={'source': 'data/constitucion-politica-col.pdf', 'page': 0}), Document(page_content='Colombia, en su vida, honra, bienes, creencias, y \ndemás derechos y libertades, y para asegurar el', metadata={'source': 'data/constitucion-politica-col.pdf', 'page': 0}), Document(page_content='siguiente:\nConstitución Política de Colombia\nTÍTULO I\nDE LOS PRINCIPIOS FUNDAMENTALES', metadata={'source': 'data/constitucion-politica-col.pdf', 'page': 0}), Document(page_content='cumplimiento de los deberes sociales del Estado y \nde los particulares.', metadata={'source': 'data/constitucion-politica-col.pdf', 'page': 0})]


## Busqueda Semantica 
Para realizar este tipo de busqueda vamos a obtener un embedding de nuestra consulta y luego vamos a realizar una busqueda en el vector store basada en embeddings...

In [38]:
v = bedrock_embeddings.embed_query("Colombia es un Estado social de derecho")
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=2)
for r in results:
    print(r.page_content)
    print('----')

[1.7109375, -0.18652344, 0.48046875, 0.43554688, 0.33007812, 0.9140625, -0.22363281, -0.0009994507, 1.3359375, -0.20703125]
Artículo 1 °. Colombia es un Estado social 
de derecho, organizado en forma de República
----
Colombia, en su vida, honra, bienes, creencias, y 
demás derechos y libertades, y para asegurar el
----


### Nota 
Existe un wrapper de LangChain que nos va a permitir realizar una busqueda semantica de forma mas sencilla

In [39]:
llm_claude = Bedrock(
  model_id="anthropic.claude-v2",
  model_kwargs={'max_tokens_to_sample': 200},
  credentials_profile_name="default", 
  region_name="us-east-1"
)

In [41]:
vector_store_faiss_wrapper = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(vector_store_faiss_wrapper.query("En Colombia hay respeto de la dignidad humana?", llm=llm_claude))

 Basado en el contexto proporcionado, se puede inferir que en Colombia hay respeto por la dignidad humana. Específicamente, el Artículo 1° de la Constitución de Colombia establece que es una república democrática, participativa y pluralista, fundada en el respeto de la dignidad humana. Además, el Artículo 95 indica que es deber de todos en Colombia acatar la Constitución y las leyes, las cuales se presumen establecen ese respeto por la dignidad humana. Por lo tanto, según esta información, en Colombia sí hay respeto por la dignidad humana.
